## imports

In [1]:
import pandas
import numpy
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
import mysql.connector
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
import joblib
import datetime
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.vector_ar.var_model import VAR

%matplotlib qt

## Database Connection and data Fetching

In [2]:
with open('../config.env') as f:
    credentials=f.read()
credentials=credentials.split(" ")
con = mysql.connector.connect(username=credentials[0],password=credentials[1],host=credentials[2],port=credentials[3],database=credentials[4])
query=con.cursor()

In [3]:
sql="SELECT * FROM total_india_cases"
query.execute(sql)
result=query.fetchall()
column=[columns[0] for columns in query.description]
india_data=pandas.DataFrame(result,columns=column)

In [4]:
india_data

,index_no,date,ordinal_date,total_confirmed,total_active,total_recovered,total_deaths,total_tested,total_vaccinated1,total_vaccinated2,...,delta7_confirmed,delta7_active,delta7_recovered,delta7_deaths,delta7_tested,delta7_vaccinated1,delta7_vaccinated2,total_other,delta_other,delta7_other
0,1,2020-01-30,737454,1,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,2,2020-02-02,737457,2,2,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
2,3,2020-02-03,737458,3,3,0,0,0,0,0,...,3,3,0,0,0,0,0,0,0,0
3,4,2020-02-14,737469,3,0,3,0,0,0,0,...,0,-3,3,0,0,0,0,0,0,0
4,5,2020-03-02,737486,5,2,3,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,575,2021-09-23,738056,33593513,293402,32840724,446399,559932709,622902393,218615633,...,212978,-39045,249839,2121,10703560,40295488,28796794,12988,13,63
575,576,2021-09-24,738057,33623078,294619,32868773,446690,561661383,627174874,221754286,...,207189,-39055,244055,2127,10881110,31083028,22537043,12996,8,62
576,577,2021-09-25,738058,33651247,296493,32894794,446949,563243245,630831085,225250442,...,204228,-28342,230424,2080,10903077,29784635,21924561,13011,15,66
577,578,2021-09-26,738059,33678246,293584,32924419,447225,564408251,633216286,226942725,...,200399,-18559,216832,2060,10786485,29549513,22041354,13018,7,66


In [5]:
sql="SELECT * FROM total_state_cases WHERE state_name='MH'"
query.execute(sql)
result=query.fetchall()
column=[columns[0] for columns in query.description]
maharashtra_data=pandas.DataFrame(result,columns=column)

In [6]:
maharashtra_data

,index_no,date,ordinal_date,state_name,total_confirmed,total_active,total_recovered,total_deaths,total_tested,total_vaccinated1,...,delta7_confirmed,delta7_active,delta7_recovered,delta7_deaths,delta7_tested,delta7_vaccinated1,delta7_vaccinated2,total_other,delta_other,delta7_other
0,52,2020-03-09,737493,MH,2,2,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
1,64,2020-03-10,737494,MH,5,5,0,0,0,0,...,5,5,0,0,0,0,0,0,0,0
2,76,2020-03-11,737495,MH,11,11,0,0,0,0,...,11,11,0,0,0,0,0,0,0,0
3,89,2020-03-12,737496,MH,14,14,0,0,0,0,...,14,14,0,0,0,0,0,0,0,0
4,102,2020-03-13,737497,MH,17,17,0,0,0,0,...,17,17,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,19701,2021-09-23,738056,MH,6534557,39191,6353079,138725,57646515,54812576,...,23032,-10151,32769,403,1116633,3782408,2486025,3562,2,11
564,19737,2021-09-24,738057,MH,6537843,38491,6357012,138776,57819385,55096837,...,22732,-9960,32292,387,1110257,3230397,2297707,3564,2,13
565,19773,2021-09-25,738058,MH,6541119,37984,6360735,138834,57992010,55412765,...,22617,-9935,32174,365,1117519,2729686,2076920,3566,2,13
566,19809,2021-09-26,738059,MH,6544325,37860,6364027,138870,58158000,55543151,...,22410,-5095,27140,352,1129524,2776872,2109193,3568,2,13


In [7]:
sql="SELECT * FROM total_district_cases WHERE district_name='Raigad'"
query.execute(sql)
result=query.fetchall()
column=[columns[0] for columns in query.description]
raigad_data=pandas.DataFrame(result,columns=column)

In [8]:
raigad_data

,index_no,date,ordinal_date,state_name,district_name,total_confirmed,total_active,total_recovered,total_deaths,delta_confirmed,...,delta7_deaths,total_vaccinated1,total_vaccinated2,delta_vaccinated1,delta_vaccinated2,delta7_vaccinated1,delta7_vaccinated2,total_other,delta_other,delta7_other
0,19918,2020-04-26,737541,MH,Raigad,57,36,20,1,0,...,0,0,0,0,0,0,0,0,0,0
1,20334,2020-04-27,737542,MH,Raigad,61,39,21,1,4,...,0,0,0,0,0,0,0,0,0,0
2,20759,2020-04-28,737543,MH,Raigad,66,44,21,1,5,...,0,0,0,0,0,0,0,0,0,0
3,21189,2020-04-29,737544,MH,Raigad,69,44,23,2,3,...,1,0,0,0,0,0,0,0,0,0
4,21623,2020-04-30,737545,MH,Raigad,71,44,24,3,2,...,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,375164,2021-09-23,738056,MH,Raigad,191843,1086,186246,4504,164,...,13,1391369,541390,6166,5187,163496,68884,7,0,0
516,375911,2021-09-24,738057,MH,Raigad,191974,1204,186259,4504,131,...,10,1394517,545160,3148,3770,98380,50105,7,0,0
517,376658,2021-09-25,738058,MH,Raigad,192123,1127,186483,4506,149,...,6,1434051,565067,39534,19907,116313,61609,7,0,0
518,377405,2021-09-26,738059,MH,Raigad,192279,1056,186710,4506,156,...,6,1439241,568103,5190,3036,114504,58907,7,0,0


## function for traintest_splitting and next 7 days ordinal_date creating

In [9]:
def splitting(data,training_ratio):
    length=len(data)
    traing_indexes=int(length*training_ratio)
    training_set=data[:traing_indexes]
    testing_set=data[traing_indexes:]
    return training_set,testing_set

In [10]:
def returnNext7days():
    date=738060
    date_range=[]
    l=[]
    for i in range(1,8):
        l.append([date+i])
        date_range.append(datetime.date.fromordinal(date+i))
    return l,date_range
predictingdate,daterange=returnNext7days()


## Train Test Splitting

In [11]:
maindata=india_data
# maindata=maharashtra_data
# maindata=raigad_data

In [12]:
maindata_labels=maindata['total_confirmed']

In [13]:
maindata

,index_no,date,ordinal_date,total_confirmed,total_active,total_recovered,total_deaths,total_tested,total_vaccinated1,total_vaccinated2,...,delta7_confirmed,delta7_active,delta7_recovered,delta7_deaths,delta7_tested,delta7_vaccinated1,delta7_vaccinated2,total_other,delta_other,delta7_other
0,1,2020-01-30,737454,1,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,2,2020-02-02,737457,2,2,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
2,3,2020-02-03,737458,3,3,0,0,0,0,0,...,3,3,0,0,0,0,0,0,0,0
3,4,2020-02-14,737469,3,0,3,0,0,0,0,...,0,-3,3,0,0,0,0,0,0,0
4,5,2020-03-02,737486,5,2,3,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,575,2021-09-23,738056,33593513,293402,32840724,446399,559932709,622902393,218615633,...,212978,-39045,249839,2121,10703560,40295488,28796794,12988,13,63
575,576,2021-09-24,738057,33623078,294619,32868773,446690,561661383,627174874,221754286,...,207189,-39055,244055,2127,10881110,31083028,22537043,12996,8,62
576,577,2021-09-25,738058,33651247,296493,32894794,446949,563243245,630831085,225250442,...,204228,-28342,230424,2080,10903077,29784635,21924561,13011,15,66
577,578,2021-09-26,738059,33678246,293584,32924419,447225,564408251,633216286,226942725,...,200399,-18559,216832,2060,10786485,29549513,22041354,13018,7,66


In [14]:
train_data,test_data=splitting(maindata,0.95)

In [15]:
# train_labels=train_data['delta_confirmed']
train_labels=train_data['total_confirmed']

In [16]:
# india_train_labels

In [17]:
train_features=train_data[['ordinal_date']]
# train_fetures=train_data[['ordinal_date','delta_tested']]

In [18]:
# train_labels

In [19]:
# test_labels=test_data['delta_confirmed']
test_labels=test_data['total_confirmed']

In [20]:
# test_labels

In [21]:
test_features=test_data[['ordinal_date']]
# test_features=test_data[['ordinal_date','delta_tested']]

In [22]:
# test_features

In [23]:
# print(predictingdate)
# print(daterange)

## Model Implementation 
1. Linear Regression
2. Random Forest Regression
3. Decision Tree Regression
4. Extra Tree Regression

In [24]:
linear_regression_model=LinearRegression()
random_forest_model=RandomForestRegressor()
decision_tree_model=DecisionTreeRegressor()
extra_tree_model=ExtraTreeRegressor()

#### Linear regression

In [25]:
linear_regression_model.fit(train_features,train_labels)

LinearRegression()

In [26]:
linear_regression_model_train_predicted=linear_regression_model.predict(train_features)

In [27]:
# linear_regression_model_train_predicted

In [28]:
linear_regression_model_test_predicted=linear_regression_model.predict(test_features)

In [29]:
# linear_regression_model_test_predicted

In [30]:
linear_regression_model_next_7_days_predicted=linear_regression_model.predict(predictingdate)

In [31]:
# linear_regression_model_next_7_days_predicted

In [32]:
pyplot.figure('linear regression')
line_of_regression=numpy.concatenate((linear_regression_model_train_predicted,linear_regression_model_test_predicted))
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(maindata['date'],line_of_regression,label='Line of Regression')
pyplot.plot(train_data['date'],linear_regression_model_train_predicted,label='Train Data')
pyplot.plot(test_data['date'],linear_regression_model_test_predicted,label='Test Data')
pyplot.plot(daterange,linear_regression_model_next_7_days_predicted,label='Future Prediction')
pyplot.legend()
pyplot.show()

#### Ramdom Forest Regression regression

In [33]:
random_forest_model.fit(train_features,train_labels)

RandomForestRegressor()

In [34]:
random_forest_model_train_predicted=random_forest_model.predict(train_features)

In [35]:
# random_forest_model_train_predicted

In [36]:
random_forest_model_test_predicted=random_forest_model.predict(test_features)

In [37]:
# random_forest_model_test_predicted

In [38]:
random_forest_model_next_7_days_predicted=random_forest_model.predict(predictingdate)

In [39]:
# random_forest_model_next_7_days_predicted

In [40]:
pyplot.figure('random forest regression')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(train_data['date'],random_forest_model_train_predicted,label='Train Data')
pyplot.plot(test_data['date'],random_forest_model_test_predicted,label='Test Data')
pyplot.plot(daterange,random_forest_model_next_7_days_predicted,label='Future Prediction')
pyplot.legend()
pyplot.show()

#### Decision Tree regression

In [41]:
decision_tree_model.fit(train_features,train_labels)

DecisionTreeRegressor()

In [42]:
decision_tree_model_train_predicted=decision_tree_model.predict(train_features)

In [43]:
# decision_tree_model_train_predicted

In [44]:
decision_tree_model_test_predicted=decision_tree_model.predict(test_features)

In [45]:
# decision_tree_model_test_predicted

In [46]:
decision_tree_model_next_y_days_predicted=decision_tree_model.predict(predictingdate)

In [47]:
# decision_tree_model_next_y_days_predicted

In [48]:
pyplot.figure('Decision tree regression')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(train_data['date'],decision_tree_model_train_predicted,label='Train Data')
pyplot.plot(test_data['date'],decision_tree_model_test_predicted,label='Test Data')
pyplot.plot(daterange,decision_tree_model_next_y_days_predicted,label='Future Prediction')
pyplot.legend()
pyplot.show()

#### Extra Tree regression

In [49]:
extra_tree_model.fit(train_features,train_labels)

ExtraTreeRegressor()

In [50]:
extra_tree_model_train_predicted=extra_tree_model.predict(train_features)

In [51]:
# extra_tree_model_train_predicted

In [52]:
extra_tree_model_test_predicted=extra_tree_model.predict(test_features)

In [53]:
# extra_tree_model_test_predicted

In [54]:
extra_tree_model_next_7_days_predicted=extra_tree_model.predict(predictingdate)

In [55]:
# extra_tree_model_next_7_days_predicted

In [56]:
pyplot.figure('Extra tree regression')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(train_data['date'],extra_tree_model_train_predicted,label='Train Data')
pyplot.plot(test_data['date'],extra_tree_model_test_predicted,label='Test Data')
pyplot.plot(daterange,extra_tree_model_next_7_days_predicted,label='Future Prediction')
pyplot.legend()
pyplot.show()

## Evaluation
    model
    train : mean_absolute_error     root_mean_squared_error
    test :  mean_absolute_error     root_mean_squared_error

In [57]:
print('linear regrassion')
print(mean_absolute_error(linear_regression_model_train_predicted,train_labels),numpy.sqrt(mean_squared_error(linear_regression_model_train_predicted,train_labels)))
print(mean_absolute_error(linear_regression_model_test_predicted,test_labels),numpy.sqrt(mean_squared_error(linear_regression_model_test_predicted,test_labels)))

linear regrassion
3274141.049883589 3840814.828458117
3872109.9887095485 3880742.300704528


In [58]:
print('random forest regrassion')
print(mean_absolute_error(random_forest_model_train_predicted,train_labels),numpy.sqrt(mean_squared_error(random_forest_model_train_predicted,train_labels)))
print(mean_absolute_error(random_forest_model_test_predicted,test_labels),numpy.sqrt(mean_squared_error(random_forest_model_test_predicted,test_labels)))

random forest regrassion
11624.221600000048 20825.927356117907
559288.6799999997 622527.1561400886


In [59]:
print('Decision Tree regression')
print(mean_absolute_error(decision_tree_model_train_predicted,train_labels),numpy.sqrt(mean_squared_error(decision_tree_model_train_predicted,train_labels)))
print(mean_absolute_error(decision_tree_model_test_predicted,test_labels),numpy.sqrt(mean_squared_error(decision_tree_model_test_predicted,test_labels)))

Decision Tree regression
0.0 0.0
536833.0 602433.3178408412


In [60]:
print('Extra Tree Regression')
print(mean_absolute_error(extra_tree_model_train_predicted,train_labels),numpy.sqrt(mean_squared_error(extra_tree_model_train_predicted,train_labels)))
print(mean_absolute_error(extra_tree_model_test_predicted,test_labels),numpy.sqrt(mean_squared_error(extra_tree_model_test_predicted,test_labels)))

Extra Tree Regression
0.0 0.0
536833.0 602433.3178408412


## Polynomial Regression

In [61]:
pyplot.scatter(train_features,train_labels)

In [62]:
def polynomial_regrassion_model(degree_of_equation=1):
    polynomial=PolynomialFeatures(degree=degree_of_equation)
    polynomial_train_data=polynomial.fit_transform(train_features.to_numpy())
    polynomial_test_data=polynomial.fit_transform(test_features.to_numpy())
    polynomial_future_test_data=polynomial.fit_transform(predictingdate)
    polynomial_regression=LinearRegression()
    polynomial_regression.fit(polynomial_train_data,train_labels)
    polynomial_regression_model_train_predicted=polynomial_regression.predict(polynomial_train_data)
    polynomial_regression_model_test_predicted=polynomial_regression.predict(polynomial_test_data)
    polynomial_regression_model_nest_7_days_predicted=polynomial_regression.predict(polynomial_future_test_data)
#     print('Train Data Prediction')
#     print(polynomial_regression_model_train_predicted)
#     print('\n\n\nTest Data Prediction')
#     print(polynomial_regression_model_test_predicted)
#     print('\n\n\nFuture Data Prediction')
#     print(polynomial_regression_model_nest_7_days_predicted)
#     print('\n\n\n')
    print('{} degree Polynomial regression'.format(degree_of_equation))
    print(mean_absolute_error(polynomial_regression_model_train_predicted,train_labels),numpy.sqrt(mean_squared_error(polynomial_regression_model_train_predicted,train_labels)))
    print(mean_absolute_error(polynomial_regression_model_test_predicted,test_labels),numpy.sqrt(mean_squared_error(polynomial_regression_model_test_predicted,test_labels)))
    pyplot.figure('{} degree Polynomial regression'.format(degree_of_equation))
    pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
    pyplot.plot(train_data['date'],polynomial_regression_model_train_predicted,label='Train Data')
    pyplot.plot(test_data['date'],polynomial_regression_model_test_predicted,label='Test Data')
    pyplot.plot(daterange,polynomial_regression_model_nest_7_days_predicted,label='Future Prediction')
    pyplot.legend()
    pyplot.show()

In [63]:
# polynomial_regrassion_model()

In [64]:
polynomial_regrassion_model(2)

2 degree Polynomial regression
1637497.4801988637 2202872.6481548846
4273244.013469827 4373012.262487211


In [65]:
polynomial_regrassion_model(3)

3 degree Polynomial regression
1637448.2441335227 2202768.3527012817
4274869.11637931 4374650.006828526


In [66]:
polynomial_regrassion_model(4)

4 degree Polynomial regression
1637400.7301136365 2202664.155690146
4276482.767241379 4376276.4928690065


In [67]:
polynomial_regrassion_model(5)

5 degree Polynomial regression
1637353.2073828124 2202560.057301186
4278096.121531519 4377902.6895006485


In [68]:
polynomial_regrassion_model(6)

6 degree Polynomial regression
1637305.654296875 2202456.058455571
4279708.999427533 4379528.419829248


# Forcasting Models

In [69]:
forcasting_data_features=maindata[['date','total_confirmed']]

forcasting_train_data_features=train_data[['date','total_confirmed']]

forcasting_test_data_features=test_data[['date','total_confirmed']]

In [70]:
forcasting_data_features['date']=pandas.to_datetime(forcasting_data_features['date'])

forcasting_train_data_features['date']=pandas.to_datetime(forcasting_train_data_features['date'])

forcasting_test_data_features['date']=pandas.to_datetime(forcasting_test_data_features['date'])

<ipython-input-70-7a0089d374bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forcasting_data_features['date']=pandas.to_datetime(forcasting_data_features['date'])
<ipython-input-70-7a0089d374bb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forcasting_train_data_features['date']=pandas.to_datetime(forcasting_train_data_features['date'])
<ipython-input-70-7a0089d374bb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [71]:
forcasting_data_features=forcasting_data_features.set_index('date')

forcasting_train_data_features=forcasting_train_data_features.set_index('date')

forcasting_test_data_features=forcasting_test_data_features.set_index('date')

In [72]:
# forcasting_data_features

In [73]:
# forcasting_train_data_features

In [74]:
# forcasting_test_data_features

## ARIMA model

In [75]:
main_arima_model = ARIMA(forcasting_data_features,order=(5,0,3))
train_arima_model = ARIMA(forcasting_train_data_features,order=(5,0,3))

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\st

In [76]:
main_arima_fit_model=main_arima_model.fit()
train_arima_fit_model=train_arima_model.fit()

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum L

In [77]:
main_arima_fit_model_predicted=main_arima_fit_model.predict()
train_arima_fit_model_predicted=train_arima_fit_model.predict()
test_arima_predicted=train_arima_fit_model.forecast(steps=len(forcasting_test_data_features))

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


In [78]:
arima_model_forcast=main_arima_fit_model.forecast(steps=7)

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


In [79]:
# main_arima_fit_model_predicted
# train_arima_fit_model_predicted
# test_arima_predicted
arima_model_forcast

579    3.370916e+07
580    3.372290e+07
581    3.373575e+07
582    3.374926e+07
583    3.375901e+07
584    3.377155e+07
585    3.377948e+07
Name: predicted_mean, dtype: float64

In [91]:
pyplot.figure('ARIMA model Forcasting')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(train_data['date'],train_arima_fit_model_predicted,label='Train Data')
pyplot.plot(test_data['date'],test_arima_predicted,label='Test Data')
pyplot.plot(daterange,arima_model_forcast,label='ARIMA Future Prediction')
pyplot.legend()
pyplot.show()

In [81]:
print(mean_absolute_error(train_arima_fit_model_predicted,train_labels),numpy.sqrt(mean_squared_error(train_arima_fit_model_predicted,train_labels)))
print(mean_absolute_error(test_arima_predicted,test_labels),numpy.sqrt(mean_squared_error(test_arima_predicted,test_labels)))
    

24102.90713138654 470398.4244600237
73662.35401032968 94393.48683920378


## SARIMA model

In [82]:
main_sarima_model=SARIMAX(forcasting_data_features)
train_sarima_model = ARIMA(forcasting_train_data_features,order=(5,0,3))

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\st

In [83]:
main_sarima_fit_model=main_sarima_model.fit()
train_sarima_fit_model=train_sarima_model.fit()

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [84]:
# sarima_model_fit.predict(590)
main_sarima_fit_model_predicted=main_sarima_fit_model.predict()
train_sarima_fit_model_predicted=train_sarima_fit_model.predict()
test_sarima_predicted=train_sarima_fit_model.forecast(steps=len(forcasting_test_data_features))

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


In [86]:
sarima_model_forcast=main_sarima_fit_model.forecast(steps=7)

C:\Users\Pankaj singh\AppData\Roaming\Python\Python39\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


In [87]:
# main_sarima_fit_model_predicted
# train_sarima_fit_model_predicted
# test_sarima_predicted
sarima_model_forcast

579    3.369283e+07
580    3.369250e+07
581    3.369218e+07
582    3.369185e+07
583    3.369153e+07
584    3.369120e+07
585    3.369087e+07
Name: predicted_mean, dtype: float64

In [92]:
pyplot.figure('SARIMA model Forcasting')
pyplot.plot(maindata['date'],maindata_labels,label='Original Data')
pyplot.plot(train_data['date'],train_sarima_fit_model_predicted,label='Train Data')
pyplot.plot(test_data['date'],test_sarima_predicted,label='Test Data')
pyplot.plot(daterange,sarima_model_forcast,label='SARIMA Future Prediction')
pyplot.legend()
pyplot.show()

In [93]:
print(mean_absolute_error(train_sarima_fit_model_predicted,train_labels),numpy.sqrt(mean_squared_error(train_sarima_fit_model_predicted,train_labels)))
print(mean_absolute_error(test_sarima_predicted,test_labels),numpy.sqrt(mean_squared_error(test_sarima_predicted,test_labels)))
    

24102.90713138654 470398.4244600237
73662.35401032968 94393.48683920378
